In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('cqx20-21data.txt')
winter2 = pd.read_csv ('cqx21-22data.txt')
winter3 = pd.read_csv ('cqx22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
# eliminate special weather
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 52]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 52]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 52]

In [8]:
precip_days1.dtypes


station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
cqx = LocationInfo(
    'Chicopee', 'Massachusetts', 
    latitude= 41.6827,
    longitude= -69.9616,
    timezone='America/Connecticut'
)

cqx

LocationInfo(name='Chicopee', region='Massachusetts', timezone='America/Connecticut', latitude=41.6827, longitude=-69.9616)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(cqx.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(cqx.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(cqx.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:48:08  2020-12-01 16:09:52  2020-12-01
1   2020-12-02 06:49:09  2020-12-02 16:09:37  2020-12-02
2   2020-12-03 06:50:10  2020-12-03 16:09:25  2020-12-03
3   2020-12-04 06:51:09  2020-12-04 16:09:15  2020-12-04
4   2020-12-05 06:52:08  2020-12-05 16:09:08  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:22:10  2021-02-24 17:24:24  2021-02-24
86  2021-02-25 06:20:38  2021-02-25 17:25:36  2021-02-25
87  2021-02-26 06:19:06  2021-02-26 17:26:48  2021-02-26
88  2021-02-27 06:17:33  2021-02-27 17:28:00  2021-02-27
89  2021-02-28 06:16:00  2021-02-28 17:29:11  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:47:53  2021-12-01 16:09:55  2021-12-01
1   2021-12-02 06:48:55  2021-12-02 16:09:40  2021-12-02
2   2021-12-03 06:49:55  2021-12-03 16:09:27  2021-12-03
3   2021-12-04 06:50:55  2021-12-04 16:09:17  2021-12-04
4   2021-12-05 06:51:54  2021-12-05 16:09:09  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:22:32  2022-02-24 17:24:06  2022-02-24
86  2022-02-25 06:21:01  2022-02-25 17:25:19  2022-02-25
87  2022-02-26 06:19:29  2022-02-26 17:26:31  2022-02-26
88  2022-02-27 06:17:56  2022-02-27 17:27:42  2022-02-27
89  2022-02-28 06:16:23  2022-02-28 17:28:54  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:47:37  2022-12-01 16:09:59  2022-12-01
1   2022-12-02 06:48:40  2022-12-02 16:09:43  2022-12-02
2   2022-12-03 06:49:41  2022-12-03 16:09:30  2022-12-03
3   2022-12-04 06:50:41  2022-12-04 16:09:19  2022-12-04
4   2022-12-05 06:51:40  2022-12-05 16:09:11  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:22:53  2023-02-24 17:23:49  2023-02-24
86  2023-02-25 06:21:23  2023-02-25 17:25:01  2023-02-25
87  2023-02-26 06:19:51  2023-02-26 17:26:13  2023-02-26
88  2023-02-27 06:18:18  2023-02-27 17:27:25  2023-02-27
89  2023-02-28 06:16:45  2023-02-28 17:28:37  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,CQX,2020-12-08 16:52:00,0.0001,-SN,2020-12-08,2020-12-08 06:54:54,2020-12-08 16:09:00
1,CQX,2020-12-09 21:52:00,0.0001,-SN,2020-12-09,2020-12-09 06:55:46,2020-12-09 16:09:02
2,CQX,2020-12-09 22:52:00,0.2500,-SN BR,2020-12-09,2020-12-09 06:55:46,2020-12-09 16:09:02
3,CQX,2020-12-09 23:52:00,0.7600,-SN BR,2020-12-09,2020-12-09 06:55:46,2020-12-09 16:09:02
4,CQX,2020-12-10 01:52:00,0.2500,-SN BR,2020-12-10,2020-12-10 06:56:37,2020-12-10 16:09:07
...,...,...,...,...,...,...,...
142,CQX,2021-02-20 13:52:00,0.2500,-SN BR,2021-02-20,2021-02-20 06:28:06,2021-02-20 17:19:32
143,CQX,2021-02-20 19:52:00,0.0001,-SN,2021-02-20,2021-02-20 06:28:06,2021-02-20 17:19:32
144,CQX,2021-02-20 20:52:00,0.0001,-SN,2021-02-20,2021-02-20 06:28:06,2021-02-20 17:19:32
145,CQX,2021-02-21 13:52:00,0.0001,-SN,2021-02-21,2021-02-21 06:26:38,2021-02-21 17:20:45


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,CQX,2021-12-19 22:52:00,0.2500,-SN BR,2021-12-19,2021-12-19 07:02:49,2021-12-19 16:11:26
1,CQX,2022-01-03 20:52:00,0.0001,-SN,2022-01-03,2022-01-03 07:07:18,2022-01-03 16:21:42
2,CQX,2022-01-03 21:52:00,0.0001,-SN,2022-01-03,2022-01-03 07:07:18,2022-01-03 16:21:42
3,CQX,2022-01-03 23:52:00,0.0001,-SN,2022-01-03,2022-01-03 07:07:18,2022-01-03 16:21:42
4,CQX,2022-01-04 02:52:00,0.0001,-SN,2022-01-04,2022-01-04 07:07:18,2022-01-04 16:22:38
...,...,...,...,...,...,...,...
89,CQX,2022-02-26 01:52:00,0.7600,-SN BR,2022-02-26,2022-02-26 06:19:29,2022-02-26 17:26:31
90,CQX,2022-02-26 02:52:00,0.2500,-SN BR,2022-02-26,2022-02-26 06:19:29,2022-02-26 17:26:31
91,CQX,2022-02-26 04:52:00,0.2500,-SN BR,2022-02-26,2022-02-26 06:19:29,2022-02-26 17:26:31
92,CQX,2022-02-26 06:52:00,0.0001,-SN,2022-02-26,2022-02-26 06:19:29,2022-02-26 17:26:31


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,CQX,2022-12-14 07:52:00,0.2500,+SN,2022-12-14,2022-12-14 06:59:23,2022-12-14 16:09:41
1,CQX,2022-12-14 08:52:00,1.2700,-SN,2022-12-14,2022-12-14 06:59:23,2022-12-14 16:09:41
2,CQX,2023-01-11 21:52:00,0.0001,-SN,2023-01-11,2023-01-11 07:06:15,2023-01-11 16:29:29
3,CQX,2023-01-11 23:52:00,0.0001,-SN,2023-01-11,2023-01-11 07:06:15,2023-01-11 16:29:29
4,CQX,2023-01-15 08:52:00,2.7900,-SN BR,2023-01-15,2023-01-15 07:04:49,2023-01-15 16:33:57
5,CQX,2023-01-15 15:52:00,1.2700,-SN BR,2023-01-15,2023-01-15 07:04:49,2023-01-15 16:33:57
6,CQX,2023-01-15 16:52:00,2.0300,-SN BR,2023-01-15,2023-01-15 07:04:49,2023-01-15 16:33:57
7,CQX,2023-01-15 18:52:00,1.7800,-SN BR,2023-01-15,2023-01-15 07:04:49,2023-01-15 16:33:57
8,CQX,2023-01-15 19:52:00,0.5100,-SN BR,2023-01-15,2023-01-15 07:04:49,2023-01-15 16:33:57
9,CQX,2023-01-15 20:52:00,3.5600,-SN BR,2023-01-15,2023-01-15 07:04:49,2023-01-15 16:33:57


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
12,CQX,2020-12-17 07:52:00,2.2900,-SN BR,2020-12-17,2020-12-17 07:01:46,2020-12-17 16:10:44
13,CQX,2020-12-17 10:52:00,0.7600,-SN BR,2020-12-17,2020-12-17 07:01:46,2020-12-17 16:10:44
21,CQX,2020-12-18 07:52:00,0.0001,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07
22,CQX,2020-12-18 08:52:00,0.2500,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07
23,CQX,2020-12-18 09:52:00,0.0001,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07
24,CQX,2020-12-18 10:52:00,0.2500,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07
25,CQX,2020-12-18 11:52:00,0.7600,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07
26,CQX,2020-12-18 12:52:00,0.5100,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07
27,CQX,2020-12-18 13:52:00,0.0001,-SN,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07
28,CQX,2020-12-18 14:52:00,0.0001,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
13,CQX,2020-12-17 10:52:00,0.7600,-SN BR,2020-12-17,2020-12-17 07:01:46,2020-12-17 16:10:44,2020-12-17 08:01:46,2020-12-17 15:10:44
22,CQX,2020-12-18 08:52:00,0.2500,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07,2020-12-18 08:02:23,2020-12-18 15:11:07
23,CQX,2020-12-18 09:52:00,0.0001,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07,2020-12-18 08:02:23,2020-12-18 15:11:07
24,CQX,2020-12-18 10:52:00,0.2500,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07,2020-12-18 08:02:23,2020-12-18 15:11:07
25,CQX,2020-12-18 11:52:00,0.7600,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07,2020-12-18 08:02:23,2020-12-18 15:11:07
26,CQX,2020-12-18 12:52:00,0.5100,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07,2020-12-18 08:02:23,2020-12-18 15:11:07
27,CQX,2020-12-18 13:52:00,0.0001,-SN,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07,2020-12-18 08:02:23,2020-12-18 15:11:07
28,CQX,2020-12-18 14:52:00,0.0001,-SN BR,2020-12-18,2020-12-18 07:02:23,2020-12-18 16:11:07,2020-12-18 08:02:23,2020-12-18 15:11:07
33,CQX,2021-01-06 10:52:00,0.0001,-SN BR,2021-01-06,2021-01-06 07:07:09,2021-01-06 16:24:48,2021-01-06 08:07:09,2021-01-06 15:24:48
34,CQX,2021-01-06 11:52:00,0.0001,-SN,2021-01-06,2021-01-06 07:07:09,2021-01-06 16:24:48,2021-01-06 08:07:09,2021-01-06 15:24:48


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.09483
2021-2022: 0.5317333333333334
2022-2023: 0.43938


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,CQX,2020-12-08 16:52:00,0.0001,-SN,2020-12-08,2020-12-08 06:54:54,2020-12-08 16:09:00
1,CQX,2020-12-09 21:52:00,0.0001,-SN,2020-12-09,2020-12-09 06:55:46,2020-12-09 16:09:02
2,CQX,2020-12-09 22:52:00,0.2500,-SN BR,2020-12-09,2020-12-09 06:55:46,2020-12-09 16:09:02
3,CQX,2020-12-09 23:52:00,0.7600,-SN BR,2020-12-09,2020-12-09 06:55:46,2020-12-09 16:09:02
4,CQX,2020-12-10 01:52:00,0.2500,-SN BR,2020-12-10,2020-12-10 06:56:37,2020-12-10 16:09:07
...,...,...,...,...,...,...,...
135,CQX,2021-02-20 02:52:00,1.7800,-SN BR,2021-02-20,2021-02-20 06:28:06,2021-02-20 17:19:32
136,CQX,2021-02-20 03:52:00,1.0200,-SN BR,2021-02-20,2021-02-20 06:28:06,2021-02-20 17:19:32
137,CQX,2021-02-20 05:52:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:28:06,2021-02-20 17:19:32
143,CQX,2021-02-20 19:52:00,0.0001,-SN,2021-02-20,2021-02-20 06:28:06,2021-02-20 17:19:32


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_28600\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.7943021276595746
2021-2022: 0.1688166666666667
2022-2023: 0.6262


In [66]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'MA'

data.to_csv('SO.cqx.csv', index=False)